## Topic model

In [77]:
import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs, pat_inv_map, get_topics)

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary, mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup
import pickle

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# TODO (Lee) - resolve deprecation warnings
import warnings; warnings.simplefilter('ignore')

In [17]:
np.random.seed(3)

### Acquire data

In [74]:
%%capture
# acquire dataset of ML patents by call to PatentsView API 
raw_data_1000 = get_ml_patents(pats_per_page=1000)

In [76]:
len(raw_data_1000[0])

47

In [72]:
# adhoc requests calls
endpoint_url = 'http://www.patentsview.org/api/patents/query'
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},
              {"_text_phrase":{"patent_abstract":"natural language"}},
              {"_text_phrase":{"patent_abstract":"machine learning"}},
              {"_text_phrase":{"patent_title":"machine learning"}},
              {"_text_phrase":{"patent_abstract":"computer vision"}},
              {"_text_phrase":{"patent_abstract":"computer vision"}}]}
pat_fields = get_patent_fields_list()
fields = pat_fields
pats_per_page = 1000
options = {"per_page": pats_per_page}
sort = [{"patent_date": "desc"}]

params = {'q': json.dumps(query),
          'f': json.dumps(fields),
          'o': json.dumps(options),
          's': json.dumps(sort)}

# request and results
response = requests.get(endpoint_url, params=params)
status = response.status_code
results = response.json()
# print(results)
count = results.get("count")
data_resp = results['patents']
total_pats = results.get("total_patent_count")
# return data_resp

In [73]:
count

1000

### Structure data

In [24]:
# specify fields (key:val pairs) to retain from full api response
retained_keys = ['patent_number', 'patent_date', 'patent_title', 'patent_abstract', 'inventors']
data = trim_data(data=raw_data, keys=retained_keys)

# create new key:value pair by combining values from patent_title and patent_abstract keys
data = create_title_abstract_col(data=data)

# create dataframe, organize columns and sort by patent_date
df = structure_dataframe(data=data)

# partition data
data_train, data_test = partition_dataframe(df, .8)

# convert dataframe to list
text_data = df.patent_title_abstract.tolist()
text_train = data_train.patent_title_abstract.tolist()
text_test = data_test.patent_title_abstract.tolist()

# TODO (Lee) - this explores direct structuring from api response without df
text_list = []
for i in data:
    text_list.append(i['patent_title_abstract'])
# text_list

### Pre-process text data

In [53]:
# uncomment to download stop words
# !python -m spacy download en
stop_words = stopwords.words('/Users/lee/Documents/techniche/techniche/data/stopwords/english')

# construct pipeline
nlp = build_pipeline()
print(nlp.pipe_names)

# pre-process documents TODO (Lee) - via text_list directly from api response
processed_docs_1 = process_docs(text_list)

# pre-process documents TODO (Lee) - via df to list
processed_docs_2 = process_docs(text_train)

### Build corpus and dictionary

# build dictionary
id_to_word = Dictionary(processed_docs_2)

# apply term-doc frequency (list of (token_id, token_count) tuples) to docs in corpus 
corpus_2 = [id_to_word.doc2bow(doc) for doc in processed_docs_2]

# view formatted corpus
formatted_corpus = [[(id_to_word[id], freq) for id, freq in text] for text in corpus]
#formatted_corpus
#id_to_word.token2id

['tagger', 'parser', 'ner']


### Train model #1: Genism LDA model

In [42]:
# TODO (Lee) - resolve deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus_2,
                     id2word=id_to_word,
                     num_topics=5, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

### Model #1 - Explore and visualize

In [43]:
# explore topics visually
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus_2, id_to_word)
# viz_topics_1

In [31]:
# keywords in n topics in corpus
pprint(model_lda.print_topics())

[(0,
  '0.019*"system" + 0.013*"documentation" + 0.013*"user" + 0.013*"embodiments" '
  '+ 0.010*"model" + 0.010*"learning" + 0.010*"machine" + 0.009*"based" + '
  '0.009*"auto" + 0.009*"text"'),
 (1,
  '0.027*"machine" + 0.026*"data" + 0.025*"learning" + 0.017*"plurality" + '
  '0.015*"classifier" + 0.014*"method" + 0.011*"systems" + 0.011*"user" + '
  '0.011*"information" + 0.010*"search"'),
 (2,
  '0.032*"content" + 0.017*"language" + 0.016*"based" + 0.013*"one" + '
  '0.013*"method" + 0.013*"natural" + 0.011*"entity" + 0.011*"first" + '
  '0.011*"machine" + 0.011*"provided"'),
 (3,
  '0.019*"recognition" + 0.015*"state" + 0.015*"crowd" + 0.013*"method" + '
  '0.013*"output" + 0.013*"electronic" + 0.013*"documents" + 0.011*"image" + '
  '0.011*"patient" + 0.009*"one"'),
 (4,
  '0.030*"one" + 0.023*"data" + 0.019*"system" + 0.016*"machine" + '
  '0.016*"learning" + 0.013*"using" + 0.013*"response" + 0.013*"distributed" + '
  '0.012*"value" + 0.012*"model"')]


In [32]:
# most important keywords, and the respective weight, that form topic with index 0
pprint(model_lda.print_topic(4))

('0.030*"one" + 0.023*"data" + 0.019*"system" + 0.016*"machine" + '
 '0.016*"learning" + 0.013*"using" + 0.013*"response" + 0.013*"distributed" + '
 '0.012*"value" + 0.012*"model"')


### Model #1 - Evaluate

As an unsupervised task, there are no labels with which to evaluate the "expected" prediction.

- Per Radhurek: "Each topic modeling method (LSI, LDA...) has ways to measure "internal quality" (perplexity, reconstruction error...), but these are an artifact of the particular approach taken (bayesian training, matrix factorization...). 
- No way to compare such scores across different types of topic models. 
- Extrinsic evaluation
    - evaluate quality of superordinate task that topic models trained for (e.g., when task is retrieval of semantically similar documents, evaluate by manually tagging a set of similar documents and then see how well a given semantic model maps those similar documents together).
    - Automatic tagging (word intrusion method) - Such manual tagging can be resource intensive, so people hae been looking for clever ways to automate it. In Reading tea leaves: How humans interpret topic models, Wallach&al suggest a "word intrusion" method that works well for models where the topics are meant to be "human interpretable", such as LDA. For each trained topic, they take its first ten words, then substitute one of them with another, randomly chosen word (intruder!) and see whether a human can reliably tell which one it was. If so, the trained topic is topically coherent (good); if not, the topic has no discernible theme (bad):

#### Model #1 - Evaluate - Coherence

Calculate topic coherence for topic models.
Implements 'CoherenceModel' coherence pipeline (segmentation, probability estimation, confirmation measure, aggregation) from Roeder et al., 2015. "Exploring the space of topic coherence measures", WSDM '15 Proceedings of the Eighth ACM International Conference on Web Search and Data Mining (WSDM) 2015, 399-408.

In [58]:
# calculate coherence metric for test_set (n = 20 docs/100 docs)
coherence = CoherenceModel(model=model_lda, texts=processed_docs_3, dictionary=id_to_word, coherence='c_v')
coherence_1_3 = coherence.get_coherence()
print(coherence_1_3)

0.41013435823858185


In [56]:
# calculate coherence metric for train set ((n = 80 docs/100 docs))
coherence = CoherenceModel(model=model_lda, texts=processed_docs_2, dictionary=id_to_word, coherence='c_v')
coherence_1_2 = coherence.get_coherence()
print(coherence_1_2)

0.4591438924438723


In [ ]:
# calculate coherence metric for each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
# print(coherence_1)

#### Model #1 - Evaluate - Perplexity

In [57]:
# pre-process test set documents TODO (Lee) - via df to list
processed_docs_3 = process_docs(text_test)

# build dictionary
id_to_word = Dictionary(processed_docs_3)

# apply term-doc frequency (list of (token_id, token_count) tuples) to docs in corpus 
corpus_3 = [id_to_word.doc2bow(doc) for doc in processed_docs_3]

In [40]:
# calculate perplexity metric 

# metric calculates and returns per-word likelihood bound using a chunk of documents as evaluation corpus
# output calculated statistics, including the perplexity=2^(-bound), to log at INFO level
# Returns the variational bound score calculated for each word

perplexity_train = model_lda.log_perplexity(corpus_2)
print(perplexity_train)

-8.987212168216839


In [41]:
perplexity_test = model_lda.log_perplexity(corpus_3)
print(perplexity_test)

-8.987212097864088


#### Model #1 - Evaluate - holdout set

In [ ]:
# TODO (Lee) - evaluate on 1k documents **not** used in LDA training
doc_stream = (tokens for _, tokens in iter_wiki('./data/simplewiki-20140623-pages-articles.xml.bz2'))  # generator
test_docs = list(itertools.islice(doc_stream, 8000, 9000))

#### Model #1 - Evaluate - Doc split

In [ ]:
# TODO (Lee) - split each document into two parts, and check that 1) topics of the first half are similar to 
topics of the second 2) halves of different documents are mostly dissimilar:

In [ ]:
# TODO (Lee)
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[id2word.doc2bow(tokens[: len(tokens) / 2])] for tokens in test_docs]
    part2 = [model[id2word.doc2bow(tokens[len(tokens) / 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between 10,000 random parts (lower is better):")    
    print(np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [ ]:
# TODO (Lee)
print("LDA results:")
intra_inter(lda_model, test_docs)

#### Model #1 - Evaluate - Log likelihood

### Model #1 - Predict

#### Model #1 - Predict - Pickle model

In [ ]:
# pickle model
pickle.dump(model_lda, open('/Users/lee/Documents/techniche/techniche/data/model_lda_1.pkl','wb'))

In [ ]:
model_lda = pickle.load(open('/Users/lee/Documents/techniche/techniche/data/model_lda_1.pkl','rb'))

#### Model #1 - inference

In [ ]:
# extracts topics for given document from Gensim
# TODO (Lee) - call from test_model.py
def get_topics(doc, k=5, model_lda=model_lda):
    topic_id = sorted(model_lda[doc][0], key=lambda x: -x[1])
    top_k_topics = [x[0] for x in topic_id[:k]]
    return [(i, model_lda.print_topic(i)) for i in top_k_topics]

In [ ]:
# `get_document_topics()` returns topic probability distribution for given document
# topic_dist_675_a = model_lda.get_document_topics(corpus[50])
# pprint(sorted(topic_dist_50_a))

In [ ]:
# topicid = 3
# model_lda.get_topic_terms(topicid, topn=10)

In [ ]:
# text_train[doc_id]
# doc_id = 675
# topic_dist_675_b = sorted(get_topics(corpus[doc_id], k=10)), text_train[doc_id]
# pprint(topic_dist_675_b)

In [ ]:
# text = 'virtual dictionary lexicon enablement voice'.split()
text_input_1 = 'smart assistant transformer model translation'.split()

In [ ]:
id_to_word.doc2bow(text_input_1)

In [ ]:
get_topics(id_to_word.doc2bow(text_input_1), k=10)

In [ ]:
text_input_2 = """At the Siri International team within Apple we bring the Siri intelligent assistant to our customers worldwide in over 40 languages and dialects. Join us, and tackle some of the most challenging problems in natural language processing and large scale applied machine learning. You will build cutting edge natural language understanding technologies and deploy them on a global scale. Your work will advance and shape the future vision of our multi-lingual, multi-cultural Siri assistant, and Search applications used by millions across the world Key Qualifications
Extensive track record of scientific research in NLP and Machine Learning, or similar experience in developing language technologies for shipping products.
Strong coding and software engineering skills in a mainstream programming language, such as Python, Java, C/C++.
Familiarity with NLP/ML tools and packages like Caffe, pyTorch, TensorFlow, Weka, scikit-learn, nltk, etc.
Practical experience building production quality applications related to natural language processing and machine learning.
In-depth knowledge of machine learning algorithms and ability to apply them in data driven natural language processing systems.
Ability to quickly prototype ideas / solutions, perform critical analysis, and use creative approaches for solving complex problems.
Attention to detail and excellent communication skills.
Description
We are looking for a highly motivated technologist with a strong background in Natural Language Processing and Machine Learning research. The ideal candidate will have a strong track record of taking research ideas to real-world applications. In this position you will apply your problem solving skills to challenges and opportunities within Siri International, which involves development of large-scale language technologies for many natural languages worldwide. The primary responsibility of this role is to conduct research and develop innovative machine learning, artificial intelligence and NLP solutions for multi-lingual conversational agents. You will have the opportunity to investigate cutting edge research methods that will improve customer experience of our products and enable our engineers to scale these technologies across a variety of natural languages. You will also provide technical leadership and experiment-driven insights for engineering teams on their machine learning modeling and data decisions. You will play a central role in defining the future technical directions of Siri International through quick prototyping, critical analysis and development of core multi-lingual NLP technologies.
Education & Experience
* PhD in Machine Learning, Statistics, Computer Science, Mathematics or related field with specialization in natural language processing and/or machine learning, OR * Masters degree in a related field with a strong academic/industrial track record. * Hands-on research experience in an academic or industrial setting.""".split()

In [ ]:
get_topics(id_to_word.doc2bow(text_input_2), k=10)

In [ ]:
# print keywords in n topics
sorted(model_lda.show_topics(), key=lambda x: x[1])

In [ ]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[1])

In [ ]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[1])

In [ ]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[0])

In [ ]:
# show_topic() returns n most important/relevant words, and their weights, that comprise given topic
pprint(model_lda.show_topic(1, topn=10))

In [ ]:
pprint(model_lda.show_topics(num_topics=5, num_words=10))

In [13]:
# From Gensim example - Alternate predict workflow - Create a new corpus, made of previously unseen documents.
# other_texts = [
#      ['computer', 'time', 'graph'],
#      ['survey', 'response', 'eps'],
#      ['human', 'system', 'computer']
#  ]
# other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
# unseen_doc = other_corpus[0]
# vector = lda[unseen_doc]  # get topic probability distribution for a document
# Update the model by incrementally training on the new corpus

### Model #2 - Mallet model

In [ ]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update path
path_mallet = '/Users/lee/Documents/techniche/techniche/data/mallet-2.0.8/bin/mallet'

In [ ]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_to_word)

In [ ]:
# topics
# pprint(model_2.show_topics(formatted=False))

In [ ]:
# TODO (Lee) - calculate coherence metric
coherence_2 = CoherenceModel(model=model_2, texts=data, dictionary=id_to_word, coherence='c_v')
coherence_2 = coherence_2.get_coherence()
# print(coherence_2)

In [ ]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

### Model #3 - Author topic model

In [ ]:
# construct inventor-to-doc mapping as df from nested inventors column in json api response
# df_inventors = json_normalize(data, record_path=['inventors'], meta=['patent_number', 'patent_date'])
# df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
# df_inventors.sort_values(by=['patent_date'])
# df_inventors.head(3)

In [ ]:
# quick visual index to patent number mapping
# for i in data:
#     print(data.index(i), i['patent_number'])

In [ ]:
# TODO (Lee) review fix to pat_inv_map, in which "patent" in mapping is idx of pat, not pat_number from api
pat2inv = pat_inv_map(data)

#### Construct author-topic model

In [ ]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         doc2author=pat2inv,
                         id2word=id_to_word)

In [ ]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

In [ ]:
# retrieve topic distribution for author using use model[name] syntax
# each topic has a probability of being expressed given the particular author, 
# but only the ones above a certain threshold are displayed

model_at['7788103-1']

In [ ]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# build mapping from inventor to patent
inv2pat = gensim.models.atmodel.construct_author2doc(pat2inv)

### Model #3 - Predict

In [ ]:
# prediction functions that take input of new text string, and predict topic distribution